## Import

In [1]:
import random
from loguru import logger

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import plotly.express as px
import open3d as o3d
from open3d.cpu.pybind.visualization import draw_geometries
import matplotlib.colors as mcolors
import copy
import pyransac3d as pyrsc
import laspy

from point_cloud_segmentation.config import RAW_DATA_DIR

from open3d.cpu.pybind.visualization import Visualizer, RenderOption

cmap = plt.get_cmap("gist_rainbow")

def visualize_geometries(geometries, colors=None, show_normal=False):
    if colors is None:
        colors = plt.get_cmap("gist_rainbow")
    # logger.info(len(pcd.points))
    visualizer: Visualizer = o3d.visualization.Visualizer()
    visualizer.create_window()

    if not isinstance(geometries, list):
        geometries = [geometries]

    for i, geom in enumerate(geometries):
        if hasattr(geom, 'paint_uniform_color'):
            color = cmap(i / len(geometries))[:3]
            geom.paint_uniform_color(color)
        visualizer.add_geometry(geom)

    vis_opt: RenderOption = visualizer.get_render_option()
    vis_opt.background_color = (0.3, 0.3, 0.3)
    vis_opt.point_show_normal = show_normal

    visualizer.run()
    visualizer.destroy_window()


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


2026-02-05 08:13:17.195 | INFO     | point_cloud_segmentation.config:<module>:11 - PROJ_ROOT path is: D:\wkd\work\point-cloud-segmentation


## Loading data

In [2]:
las = laspy.read(RAW_DATA_DIR / "wkd-house_1_8xds.laz")
# las = laspy.read(RAW_DATA_DIR / "wkd-house_2_8xds.laz")
print(np.unique(las.classification))

points = np.vstack((las.x, las.y, las.z)).transpose()

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)

print(pcd)


[0]
PointCloud with 70775329 points.


## Downsampling

In [3]:
voxel_size = 0.05
ds_pcd = pcd.voxel_down_sample(voxel_size=voxel_size)
print(ds_pcd)

PointCloud with 524347 points.


In [4]:
# visualize_geometries(ds_pcd)

## Segmentation

In [5]:
segmented_objects = []
unsegmented_objects = []
object_n = 2

threshold = 0.1 # how far each point to be included
# threshold = voxel_size # how far each point to be included
iterations = 1000
# og_object = copy.deepcopy(pcd)
ds_pcd_np = np.array(ds_pcd.points)
processed_pcd = copy.deepcopy(ds_pcd)
print(ds_pcd_np)
print(ds_pcd_np.shape)
for i in range(object_n):
    # logger.info("Start plane RANSAC")
    eq, idx_inliers = processed_pcd.segment_plane(distance_threshold=threshold, ransac_n=3 ,num_iterations=iterations)
    # logger.info("Finish plane RANSAC")
    processed_pcd_np = np.array(processed_pcd.points)

    # logger.info("Start getting inliers and outliers")
    inliers = processed_pcd_np[idx_inliers]

    pcd_np = np.array(processed_pcd.points)

    mask = np.ones(len(pcd_np), dtype=bool)
    mask[idx_inliers] = False
    outliers = pcd_np[mask]
    # logger.info("Finish getting inliers and outliers")

    # logger.info("Start converting np to o3d pcd")
    outliers_pcd = o3d.geometry.PointCloud()
    outliers_pcd.points = o3d.utility.Vector3dVector(outliers)
    processed_pcd = copy.deepcopy(outliers_pcd)
    # visualize_geometries(outliers_pcd)

    inlier_pcd = o3d.geometry.PointCloud()
    inlier_pcd.points = o3d.utility.Vector3dVector(inliers)
    segmented_objects.append(inlier_pcd)
    print(inliers.shape)
    print(outliers.shape)
    # logger.info("Finish converting np to o3d pcd")
    if i >= object_n - 1:
        print("ASDASD")
        unsegmented_objects.append(outliers_pcd)

print(segmented_objects)
print(unsegmented_objects)
visualize_geometries(segmented_objects)
visualize_geometries(unsegmented_objects)
# visualize_geometries(segmented_objects, unsegmented_objects)


[[ 8.43475     7.008375    4.25775   ]
 [11.64196765  9.17401294 -1.9348586 ]
 [ 6.57854078 10.80543204 -1.2858835 ]
 ...
 [-0.448      10.8905      1.0705    ]
 [-0.3185     10.6125      1.1255    ]
 [-0.4195     11.074      -0.763     ]]
(524347, 3)
(33270, 3)
(491077, 3)
(37096, 3)
(453981, 3)
ASDASD
[PointCloud with 33270 points., PointCloud with 37096 points.]
[PointCloud with 453981 points.]
